In [7]:
import sqlite3
import pandas as pd
import numpy as np

from nba_api.stats.endpoints import playbyplayv2

connection = sqlite3.connect("../Data/basketball.sqlite")

tables = pd.read_sql("""SELECT *
                        FROM sqlite_master
                        WHERE type='table';""", connection)

tables

def read_data(table, con=connection):
    return pd.read_sql(f"""SELECT * FROM {table}""", con)

all_games = read_data('Game')

all_games


,GAME_ID,SEASON_ID,TEAM_ID_HOME,TEAM_ABBREVIATION_HOME,TEAM_NAME_HOME,GAME_DATE,MATCHUP_HOME,WL_HOME,MIN_HOME,FGM_HOME,FGA_HOME,FG_PCT_HOME,FG3M_HOME,FG3A_HOME,FG3_PCT_HOME,FTM_HOME,FTA_HOME,FT_PCT_HOME,OREB_HOME,DREB_HOME,REB_HOME,AST_HOME,STL_HOME,BLK_HOME,TOV_HOME,PF_HOME,PTS_HOME,PLUS_MINUS_HOME,VIDEO_AVAILABLE_HOME,TEAM_ID_AWAY,TEAM_ABBREVIATION_AWAY,TEAM_NAME_AWAY,MATCHUP_AWAY,WL_AWAY,MIN_AWAY,FGM_AWAY,FGA_AWAY,FG_PCT_AWAY,FG3M_AWAY,FG3A_AWAY,...,PTS_OT9_HOME,PTS_OT10_HOME,PTS_HOME_y,TEAM_CITY_NAME_AWAY,TEAM_NICKNAME_AWAY,TEAM_WINS_LOSSES_AWAY,PTS_QTR1_AWAY,PTS_QTR2_AWAY,PTS_QTR3_AWAY,PTS_QTR4_AWAY,PTS_OT1_AWAY,PTS_OT2_AWAY,PTS_OT3_AWAY,PTS_OT4_AWAY,PTS_OT5_AWAY,PTS_OT6_AWAY,PTS_OT7_AWAY,PTS_OT8_AWAY,PTS_OT9_AWAY,PTS_OT10_AWAY,LAST_GAME_ID,LAST_GAME_DATE_EST,LAST_GAME_HOME_TEAM_ID,LAST_GAME_HOME_TEAM_CITY,LAST_GAME_HOME_TEAM_NAME,LAST_GAME_HOME_TEAM_ABBREVIATION,LAST_GAME_HOME_TEAM_POINTS,LAST_GAME_VISITOR_TEAM_ID,LAST_GAME_VISITOR_TEAM_CITY,LAST_GAME_VISITOR_TEAM_NAME,LAST_GAME_VISITOR_TEAM_CITY1,LAST_GAME_VISITOR_TEAM_POINTS,HOME_TEAM_WINS,HOME_TEAM_LOSSES,SERIES_LEADER,VIDEO_AVAILABLE_FLAG,PT_AVAILABLE,PT_XYZ_AVAILABLE,HUSTLE_STATUS,HISTORICAL_STATUS
0,0024600001,21946,1610610035,HUS,Toronto Huskies,1946-11-01,HUS vs. NYK,L,0,25.0,None,NaN,None,None,NaN,16.0,29.0,0.552,None,None,None,None,None,None,None,NaN,66,-2,0,1610612752,NYK,New York Knicks,NYK @ HUS,W,0,24.0,None,NaN,None,None,...,None,None,66.0,New York,Knicks,-,None,None,None,None,24,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,3.0,3.0,Tied,0.0,0.0,0.0,0.0,1.0
1,0024600003,21946,1610610034,BOM,St. Louis Bombers,1946-11-02,BOM vs. PIT,W,0,20.0,59.0,0.339,None,None,NaN,16.0,NaN,NaN,None,None,None,None,None,None,None,21.0,56,5,0,1610610031,PIT,Pittsburgh Ironmen,PIT @ BOM,L,0,16.0,72.0,0.222,None,None,...,None,None,56.0,Pittsburgh,Ironmen,-,5,15,17,14,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,5.0,1.0,St. Louis,0.0,0.0,0.0,0.0,1.0
2,0024600004,21946,1610610025,CHS,Chicago Stags,1946-11-02,CHS vs. NYK,W,0,21.0,None,NaN,None,None,NaN,21.0,NaN,NaN,None,None,None,None,None,None,None,20.0,63,16,0,1610612752,NYK,New York Knicks,NYK @ CHS,L,0,16.0,None,NaN,None,None,...,None,None,63.0,New York,Knicks,-,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,3.0,3.0,Tied,0.0,0.0,0.0,0.0,1.0
3,0024600002,21946,1610610032,PRO,Providence Steamrollers,1946-11-02,PRO vs. BOS,W,0,21.0,None,NaN,None,None,NaN,17.0,NaN,NaN,None,None,None,None,None,None,None,NaN,59,6,0,1610612738,BOS,Boston Celtics,BOS @ PRO,L,0,21.0,None,NaN,None,None,...,None,None,59.0,Boston,Celtics,-,10.0,16,14,13,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,5.0,1.0,Providence,0.0,0.0,0.0,0.0,1.0
4,0024600005,21946,1610610028,DEF,Detroit Falcons,1946-11-02,DEF vs. WAS,L,0,10.0,None,NaN,None,None,NaN,13.0,NaN,NaN,None,None,None,None,None,None,None,NaN,33,-17,0,1610610036,WAS,Washington Capitols,WAS @ DEF,W,0,18.0,None,NaN,None,None,...,None,None,33.0,Washington,Capitols,-,21,4,12,13,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,1.0,5.0,Washington,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62443,0022001076,22020,1610612757,POR,Portland Trail Blazers,2021-05-16,POR vs. DEN,W,240,46.0,87,0.529,18,43,0.419,22.0,26.0,0.846,11,40,51,24,3,6,14,16.0,132,16,1,1610612743,DEN,Denver Nuggets,DEN @ POR,L,240,44.0,98,0.449,14,37,...,0,0,NaN,Denver,Nuggets,47-25,22,33,29,32,0,0,0,0,0,0,0,0,0,0,002200089

In [14]:
all_games['GAME_DATE'] = pd.to_datetime(all_games['GAME_DATE'])

all_games['season_end_year'] = all_games['SEASON_ID'].apply(lambda x: int(x[1:]))

In [15]:
recent_games = all_games[all_games['season_end_year'].isin([2015,2016,2017,2018,2019,2020])]

In [16]:
recent_games

,GAME_ID,SEASON_ID,TEAM_ID_HOME,TEAM_ABBREVIATION_HOME,TEAM_NAME_HOME,GAME_DATE,MATCHUP_HOME,WL_HOME,MIN_HOME,FGM_HOME,FGA_HOME,FG_PCT_HOME,FG3M_HOME,FG3A_HOME,FG3_PCT_HOME,FTM_HOME,FTA_HOME,FT_PCT_HOME,OREB_HOME,DREB_HOME,REB_HOME,AST_HOME,STL_HOME,BLK_HOME,TOV_HOME,PF_HOME,PTS_HOME,PLUS_MINUS_HOME,VIDEO_AVAILABLE_HOME,TEAM_ID_AWAY,TEAM_ABBREVIATION_AWAY,TEAM_NAME_AWAY,MATCHUP_AWAY,WL_AWAY,MIN_AWAY,FGM_AWAY,FGA_AWAY,FG_PCT_AWAY,FG3M_AWAY,FG3A_AWAY,...,PTS_OT10_HOME,PTS_HOME_y,TEAM_CITY_NAME_AWAY,TEAM_NICKNAME_AWAY,TEAM_WINS_LOSSES_AWAY,PTS_QTR1_AWAY,PTS_QTR2_AWAY,PTS_QTR3_AWAY,PTS_QTR4_AWAY,PTS_OT1_AWAY,PTS_OT2_AWAY,PTS_OT3_AWAY,PTS_OT4_AWAY,PTS_OT5_AWAY,PTS_OT6_AWAY,PTS_OT7_AWAY,PTS_OT8_AWAY,PTS_OT9_AWAY,PTS_OT10_AWAY,LAST_GAME_ID,LAST_GAME_DATE_EST,LAST_GAME_HOME_TEAM_ID,LAST_GAME_HOME_TEAM_CITY,LAST_GAME_HOME_TEAM_NAME,LAST_GAME_HOME_TEAM_ABBREVIATION,LAST_GAME_HOME_TEAM_POINTS,LAST_GAME_VISITOR_TEAM_ID,LAST_GAME_VISITOR_TEAM_CITY,LAST_GAME_VISITOR_TEAM_NAME,LAST_GAME_VISITOR_TEAM_CITY1,LAST_GAME_VISITOR_TEAM_POINTS,HOME_TEAM_WINS,HOME_TEAM_LOSSES,SERIES_LEADER,VIDEO_AVAILABLE_FLAG,PT_AVAILABLE,PT_XYZ_AVAILABLE,HUSTLE_STATUS,HISTORICAL_STATUS,season_end_year
55379,0021500003,22015,1610612744,GSW,Golden State Warriors,2015-10-27,GSW vs. NOP,W,240,41.0,96,0.427,9,30,0.300,20.0,22.0,0.909,21,35,56,29,8,7,20,29.0,111,16,1,1610612740,NOP,New Orleans Pelicans,NOP @ GSW,L,240,35.0,83,0.422,6,18,...,0,111.0,New Orleans,Pelicans,0-1,35,14,26,20,0,0,0,0,0,0,0,0,0,0,0041400144,2015-04-25T00:00:00,1610612744,Golden State,Warriors,GSW,109,1610612740,New Orleans,Pelicans,NOP,98,3.0,0.0,Golden State,1.0,1.0,1.0,1.0,0.0,2015
55380,0021500002,22015,1610612741,CHI,Chicago Bulls,2015-10-27,CHI vs. CLE,W,240,37.0,87,0.425,7,19,0.368,16.0,23.0,0.696,7,40,47,13,6,10,13,22.0,97,2,1,1610612739,CLE,Cleveland Cavaliers,CLE @ CHI,L,240,38.0,94,0.404,9,29,...,0,97.0,Cleveland,Cavaliers,0-1,17,23,28,27,0,0,0,0,0,0,0,0,0,0,0041400216,2015-05-14T00:00:00,1610612741,Chicago,Bulls,CHI,73,1610612739,Cleveland,Cavaliers,CLE,94,3.0,1.0,Chicago,1.0,1.0,1.0,0.0,0.0,2015
55381,0021500001,22015,1610612737,ATL,Atlanta Hawks,2015-10-27,ATL vs. DET,L,240,37.0,82,0.451,8,27,0.296,12.0,15.0,0.800,7,33,40,22,9,4,15,25.0,94,-12,1,1610612765,DET,Detroit Pistons,DET @ ATL,W,240,37.0,96,0.385,12,29,...,0,94.0,Detroit,Pistons,1-0,25,23,34,24,0,0,0,0,0,0,0,0,0,0,0011500108,2015-10-23T00:00:00,1610612737,Atlanta,Hawks,ATL,87,1610612765,Detroit,Pistons,DET,115,3.0,1.0,Atlanta,1.0,1.0,1.0,0.0,0.0,2015
55382,0021500017,22015,1610612747,LAL,Los Angeles Lakers,2015-10-28,LAL vs. MIN,L,240,35.0,90,0.389,9,34,0.265,32.0,35.0,0.914,12,36,48,18,2,4,14,29.0,111,-1,1,1610612750,MIN,Minnesota Timberwolves,MIN @ LAL,W,240,38.0,83,0.458,5,20,...,0,111.0,Minnesota,Timberwolves,1-0,22,29,37,24,0,0,0,0,0,0,0,0,0,0,0021401186,2015-04-10T00:00:00,1610612747,Los Angeles,Lakers,LAL,106,1610612750,Minnesota,Timberwolves,MIN,98,1.0,2.0,Minnesota,1.0,1.0,1.0,1.0,0.0,2015
55383,0021500014,22015,1610612756,PHX,Phoenix Suns,2015-10-28,PHX vs. DAL,L,240,34.0,87,0.391,6,24,0.250,21.0,33.0,0.636,13,40,53,15,3,3,18,30.0,95,-16,1,1610612742,DAL,Dallas Mavericks,DAL @ PHX,W,240,40.0,85,0.471,10,21,...,0,95.0,Dallas,Mavericks,1-0,24,30,34,23,0,0,0,0,0,0,0,0,0,0,0011500094,2015-10-21T00:00:00,1610612756,Phoenix,Suns,PHX,99,1610612742,Dallas,Mavericks,DAL,87,0.0,3.0,Dallas,1.0,1.0,1.0,0.0,0.0,2015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62443,0022001076,22020,1610612757,POR,Portland Trail Blazers,2021-05-16,POR vs. DEN,W,240,46.0,87,0.529,18,43,0.419,22.0,26.0,0.846,11,40,51,24,3,6,14,16.0,132,16,1,1610612743,DEN,Denver Nuggets,DEN @ POR,L,240,44.0,98,0.449,14,37,...,0,NaN,Denver,Nuggets,47-25,22,33,29,32,0,0,0,0,0,0,0,0,0,0,00

In [21]:
from tqdm import tqdm
from time import sleep
pbp = []
for game_id in tqdm(recent_games.GAME_ID):
    pbp.append(playbyplayv2.PlayByPlayV2(game_id))
    sleep(0.1)

100%|█| 7069/7069 [2:55:02<00:00,  1.


In [27]:
pbp_dfs = []

In [28]:
for i in range(len(pbp)):
    pbp_dfs.append(pbp[i].get_data_frames()[0])

In [29]:
len(pbp_dfs)

7069

In [30]:
len(pbp)

7069

In [32]:
df = pd.concat(pbp_dfs)
df.to_csv('../Data/recent_games_pbp.csv', index=False)

In [6]:
test_game_pbp.get_data_frames()[0]

,GAME_ID,EVENTNUM,EVENTMSGTYPE,EVENTMSGACTIONTYPE,PERIOD,WCTIMESTRING,PCTIMESTRING,HOMEDESCRIPTION,NEUTRALDESCRIPTION,VISITORDESCRIPTION,SCORE,SCOREMARGIN,PERSON1TYPE,PLAYER1_ID,PLAYER1_NAME,PLAYER1_TEAM_ID,PLAYER1_TEAM_CITY,PLAYER1_TEAM_NICKNAME,PLAYER1_TEAM_ABBREVIATION,PERSON2TYPE,PLAYER2_ID,PLAYER2_NAME,PLAYER2_TEAM_ID,PLAYER2_TEAM_CITY,PLAYER2_TEAM_NICKNAME,PLAYER2_TEAM_ABBREVIATION,PERSON3TYPE,PLAYER3_ID,PLAYER3_NAME,PLAYER3_TEAM_ID,PLAYER3_TEAM_CITY,PLAYER3_TEAM_NICKNAME,PLAYER3_TEAM_ABBREVIATION,VIDEO_AVAILABLE_FLAG
0,0022001077,2,12,0,1,9:12 PM,12:00,None,Start of 1st Period (9:12 PM EST),None,None,None,0,0,None,NaN,None,None,None,0,0,None,NaN,None,None,None,0,0,None,NaN,None,None,None,0
1,0022001077,4,10,0,1,9:12 PM,12:00,Jump Ball Jones vs. Gobert: Tip to O'Neale,None,None,None,None,4,1627745,Damian Jones,1.610613e+09,Sacramento,Kings,SAC,5,203497,Rudy Gobert,1.610613e+09,Utah,Jazz,UTA,5,1626220,Royce O'Neale,1.610613e+09,Utah,Jazz,UTA,1
2,0022001077,7,5,40,1,9:12 PM,11:48,None,None,Bogdanovic Out of Bounds Lost Ball Turnover (P...,None,None,5,202711,Bojan Bogdanovic,1.610613e+09,Utah,Jazz,UTA,0,0,None,NaN,None,None,None,1,0,None,NaN,None,None,None,1
3,0022001077,8,2,1,1,9:12 PM,11:32,MISS Jones 25' 3PT Jump Shot,None,None,None,None,4,1627745,Damian Jones,1.610613e+09,Sacramento,Kings,SAC,0,0,None,NaN,None,None,None,0,0,None,NaN,None,None,None,1
4,0022001077,9,4,0,1,9:12 PM,11:27,None,None,Ingles REBOUND (Off:0 Def:1),None,None,5,204060,Joe Ingles,1.610613e+09,Utah,Jazz,UTA,0,0,None,NaN,None,None,None,0,0,None,NaN,None,None,None,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
439,0022001077,642,3,11,4,11:16 PM,0:14,None,None,Azubuike Free Throw 1 of 2 (3 PTS),120 - 96,-24,5,1628962,Udoka Azubuike,1.610613e+09,Utah,Jazz,UTA,0,0,None,NaN,None,None,None,0,0,None,NaN,None,None,None,1
440,0022001077,643,3,12,4,11:16 PM,0:14,None,None,Azubuike Free Throw 2 of 2 (4 PTS),121 - 96,-25,5,1628962,Udoka Azubuike,1.610613e+09,Utah,Jazz,UTA,0,0,None,NaN,None,None,None,0,0,None,NaN,None,None,None,1
441,0022001077,644,1,79,4,11:17 PM,0:00,King 28' 3PT Pullup Jump Shot (12 PTS) (Guy 5 ...,None,None,121 - 99,-22,4,1629663,Louis King,1.610613e+09,Sacramento,Kings,SAC,4,1629657,Kyle Guy,1.610613e+09,Sacramento,Kings,SAC,0,0,None,NaN,None,None,None,1
442,0022001077,646,18,0,4,11:17 PM,0:00,None,Instant Replay4th Period (11:17 PM EST),None,121 - 99,-22,0,644,None,NaN,None,None,None,0,0,None,NaN,None,None,None,1,0,None,NaN,None,None,None,0
